In [1]:
import numpy as np
import pandas as pd
import ast


def make_labels(row):
    label = row['synthetic'] * 4
    label += bool(row['target_labels_positions'].get('B-discount', 0))
    label += bool(row['target_labels_positions'].get('B-value', 0))
    return label


df_tt = pd.read_csv('train_data.csv', sep=',').sample(482)  # случайные исходные данные
df_tm = pd.read_csv('train_merged.csv', sep=',')

fake_train = pd.read_csv('fake_train_add.csv', sep=',')
fake_test = pd.read_csv('fake_test_add.csv', sep=',')
df_fake = pd.concat([fake_train, fake_test], ignore_index=True)

df_fake2 = pd.read_csv('fake_text2.csv', sep=',')

# проставление меток: синтетика или нет
for df in (df_tm, df_tt):
    df['synthetic'] = 0
for df in (df_fake, df_fake2):
    df['synthetic'] = 1

for df in (df_tm, df_tt, df_fake, df_fake2):
    # Преобразование значения в поле 'target_labels_positions' в словарь с помощью модуля ast
    df['target_labels_positions'] = df['target_labels_positions'].map(ast.literal_eval)
    df['row_label'] = df.apply(lambda row: make_labels(row), axis=1)

train = df_tm.iloc[:3118]
test = df_tm.iloc[-400:]

test.to_csv(f'train_merged_-400.csv', index=False)

fta = 2500
ft0 = int(fta * 0.74)
ft1 = fta - ft0
fake_train = pd.concat([df_fake[df_fake.row_label < 5].iloc[:ft0],
                        df_fake[df_fake.row_label > 4].iloc[:ft1]])
ftb = 200
ft2 = int(ftb * 0.74)
ft3 = ftb - ft2
fake_test = pd.concat([df_fake[df_fake.row_label < 5].iloc[ft0:ft0 + ft2],
                       df_fake[df_fake.row_label > 4].iloc[ft1:ft1 + ft3]])

fta = 1000
ft0 = int(fta * 0.74)
ft1 = fta - ft0
fake_train2 = pd.concat([df_fake2[df_fake2.row_label < 5].iloc[:ft0],
                         df_fake2[df_fake2.row_label > 4].iloc[:ft1]])
ftb = 200
ft2 = int(ftb * 0.74)
ft3 = ftb - ft2
fake_test2 = pd.concat([df_fake2[df_fake2.row_label < 5].iloc[ft0:ft0 + ft2],
                        df_fake2[df_fake2.row_label > 4].iloc[ft1:ft1 + ft3]])

print(train.shape, test.shape)

# добавляем случайные записи
train = pd.concat([train, df_tt], ignore_index=True)
print(train.shape, test.shape)

train = pd.concat([train, fake_train, fake_train2], ignore_index=True)
test = pd.concat([test, fake_test, fake_test2], ignore_index=True)

for df in (train, test):
    df['row_label'] = df.apply(lambda row: make_labels(row), axis=1)
    print(df['row_label'].value_counts())

df = pd.concat([train, test], ignore_index=True)

print(train.shape, test.shape, df.shape)

train_version = 8

train.to_csv(f'train_extend_{train_version}.csv', index=False)
test.to_csv(f'test_extend_{train_version}.csv', index=False)

df.to_csv(f'train_test_extend_{train_version}.csv', index=False)

print(df['row_label'].value_counts())

print('\ndf.isna().sum():')
print(df.isna().sum())

(3118, 4) (400, 4)
(3600, 4) (400, 4)
0    2723
4    1853
2     723
6     652
1     154
Name: row_label, dtype: int64
0    296
4    148
2     86
6     52
1     18
Name: row_label, dtype: int64
(6105, 4) (600, 4) (6705, 4)
0    3019
4    2001
2     809
6     704
1     172
Name: row_label, dtype: int64

df.isna().sum():
processed_text             0
target_labels_positions    0
synthetic                  0
row_label                  0
dtype: int64
